## Deep learning on Be heart Smart

In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

In [7]:
# Read the data to a dataframe
cardio_df = pd.read_csv("cardio_data_cleaned.csv")  
cardio_df.head()

,Age,gender,height,weight,systolic_bp,diastolic_bp,cholesterol,glucose,smoker,alcohol_intake,active,cardio
0,51,1,171,29.0,110.0,70.0,2,1,0,0,1,1
1,49,1,160,30.0,120.0,80.0,1,1,0,0,1,1
2,58,1,143,30.0,103.0,61.0,2,1,0,0,1,0
3,47,2,170,31.0,150.0,90.0,2,2,0,0,1,1
4,42,1,146,32.0,100.0,70.0,1,1,0,0,0,0


In [8]:
# One-hot encode the data using pd.get_dummies
cardio_df = pd.get_dummies(cardio_df, columns =["gender", "cholesterol","glucose"])
cardio_df.head()

,Age,height,weight,systolic_bp,diastolic_bp,smoker,alcohol_intake,active,cardio,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,glucose_1,glucose_2,glucose_3
0,51,171,29.0,110.0,70.0,0,0,1,1,1,0,0,1,0,1,0,0
1,49,160,30.0,120.0,80.0,0,0,1,1,1,0,1,0,0,1,0,0
2,58,143,30.0,103.0,61.0,0,0,1,0,1,0,0,1,0,1,0,0
3,47,170,31.0,150.0,90.0,0,0,1,1,0,1,0,1,0,0,1,0
4,42,146,32.0,100.0,70.0,0,0,0,0,1,0,1,0,0,1,0,0


In [9]:
column_names = ["Age","gender_1","gender_2","height","weight","systolic_bp","diastolic_bp",
                "smoker","alcohol_intake","active","cholesterol_1","cholesterol_2",
                "cholesterol_3","glucose_1","glucose_2","glucose_3", "cardio" ]
cardio_df= cardio_df.reindex(columns=column_names)
cardio_df.head()

,Age,gender_1,gender_2,height,weight,systolic_bp,diastolic_bp,smoker,alcohol_intake,active,cholesterol_1,cholesterol_2,cholesterol_3,glucose_1,glucose_2,glucose_3,cardio
0,51,1,0,171,29.0,110.0,70.0,0,0,1,0,1,0,1,0,0,1
1,49,1,0,160,30.0,120.0,80.0,0,0,1,1,0,0,1,0,0,1
2,58,1,0,143,30.0,103.0,61.0,0,0,1,0,1,0,1,0,0,0
3,47,0,1,170,31.0,150.0,90.0,0,0,1,0,1,0,0,1,0,1
4,42,1,0,146,32.0,100.0,70.0,0,0,0,1,0,0,1,0,0,0


In [10]:
cardio_df.rename(columns = {"gender_1":"Female", "gender_2":"Male", "cardio":"cardio_disease"}, inplace=True)   

In [11]:
cardio_df.head()

,Age,Female,Male,height,weight,systolic_bp,diastolic_bp,smoker,alcohol_intake,active,cholesterol_1,cholesterol_2,cholesterol_3,glucose_1,glucose_2,glucose_3,cardio_disease
0,51,1,0,171,29.0,110.0,70.0,0,0,1,0,1,0,1,0,0,1
1,49,1,0,160,30.0,120.0,80.0,0,0,1,1,0,0,1,0,0,1
2,58,1,0,143,30.0,103.0,61.0,0,0,1,0,1,0,1,0,0,0
3,47,0,1,170,31.0,150.0,90.0,0,0,1,0,1,0,0,1,0,1
4,42,1,0,146,32.0,100.0,70.0,0,0,0,1,0,0,1,0,0,0


In [12]:
cardio_df.shape

(68297, 17)

In [13]:
cardio_df.dtypes

Age                 int64
Female              uint8
Male                uint8
height              int64
weight            float64
systolic_bp       float64
diastolic_bp      float64
smoker              int64
alcohol_intake      int64
active              int64
cholesterol_1       uint8
cholesterol_2       uint8
cholesterol_3       uint8
glucose_1           uint8
glucose_2           uint8
glucose_3           uint8
cardio_disease      int64
dtype: object

In [15]:
# Split our preprocessed data into our features and target arrays
X = cardio_df.drop(["cardio_disease"],1).values
y = cardio_df["cardio_disease"].values
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51222, 14)
(17075, 14)
(51222,)
(17075,)


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25
hidden_nodes_layer4 = 12
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1700      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 12)                312       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 13        
Total params: 8,350
Trainable params: 8,350
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
1601/1601 [==============================] - 2s 825us/step - loss: 0.5538 - accuracy: 0.7261
Epoch 2/100
1601/1601 [==============================] - 1s 827us/step - loss: 0.5461 - accuracy: 0.7341
Epoch 3/100
1601/1601 [==============================] - 1s 866us/step - loss: 0.5449 - accuracy: 0.7341
Epoch 4/100
1601/1601 [==============================] - 1s 839us/step - loss: 0.5436 - accuracy: 0.7346
Epoch 5/100
1601/1601 [==============================] - 1s 829us/step - loss: 0.5428 - accuracy: 0.7350
Epoch 6/100
1601/1601 [==============================] - 1s 829us/step - loss: 0.5420 - accuracy: 0.7366
Epoch 7/100
1601/1601 [==============================] - 1s 833us/step - loss: 0.5416 - accuracy: 0.7353
Epoch 8/100
1601/1601 [==============================] - 1s 837us/step - loss: 0.5413 - accuracy: 0.7361
Epoch 9/100
1601/1601 [==============================] - 1s 838us/step - loss: 0.5405 - accuracy: 0.7367
Epoch 10/100
1601/1601 [==============================]

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {(model_loss):.3f}, Accuracy: {(model_accuracy):.3f}")

534/534 - 0s - loss: 0.6296 - accuracy: 0.7114
Loss: 0.630, Accuracy: 0.711


In [29]:
print(f"The accuracy is 71% after adding 4 hidden layers")

The accuracy is 71% after adding 4 hidden layers


## Trial 2

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 25
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 200)               3400      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 26        
Total params: 29,851
Trainable params: 29,851
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1601/1601 [==============================] - 2s 935us/step - loss: 0.5540 - accuracy: 0.7287
Epoch 2/100
1601/1601 [==============================] - 2s 965us/step - loss: 0.5478 - accuracy: 0.7322
Epoch 3/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5458 - accuracy: 0.7329
Epoch 4/100
1601/1601 [==============================] - 1s 933us/step - loss: 0.5447 - accuracy: 0.7337
Epoch 5/100
1601/1601 [==============================] - 2s 962us/step - loss: 0.5436 - accuracy: 0.7347
Epoch 6/100
1601/1601 [==============================] - 1s 930us/step - loss: 0.5430 - accuracy: 0.7360
Epoch 7/100
1601/1601 [==============================] - 2s 957us/step - loss: 0.5418 - accuracy: 0.7358
Epoch 8/100
1601/1601 [==============================] - 2s 972us/step - loss: 0.5416 - accuracy: 0.7367
Epoch 9/100
1601/1601 [==============================] - 2s 960us/step - loss: 0.5410 - accuracy: 0.7356
Epoch 10/100
1601/1601 [==============================] -

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {(model_loss):.3f}, Accuracy: {(model_accuracy):.3f}")

534/534 - 0s - loss: 0.8117 - accuracy: 0.7116
Loss: 0.812, Accuracy: 0.712


In [34]:
print(f"The accuracy is 71% after adding additional neurons to the 4 hidden layers")

The accuracy is 71% after adding additional neurons to the 4 hidden layers


### Trial 3

In [14]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cardio_df = cardio_df.drop(["height","Age"], axis =1)
cardio_df.head(10)

,Female,Male,weight,systolic_bp,diastolic_bp,smoker,alcohol_intake,active,cholesterol_1,cholesterol_2,cholesterol_3,glucose_1,glucose_2,glucose_3,cardio_disease
0,1,0,29.0,110.0,70.0,0,0,1,0,1,0,1,0,0,1
1,1,0,30.0,120.0,80.0,0,0,1,1,0,0,1,0,0,1
2,1,0,30.0,103.0,61.0,0,0,1,0,1,0,1,0,0,0
3,0,1,31.0,150.0,90.0,0,0,1,0,1,0,0,1,0,1
4,1,0,32.0,100.0,70.0,0,0,0,1,0,0,1,0,0,0
5,1,0,32.0,130.0,80.0,0,0,0,1,0,0,0,1,0,0
6,1,0,32.0,110.0,60.0,0,0,1,1,0,0,1,0,0,0
7,1,0,33.0,100.0,60.0,0,0,1,1,0,0,1,0,0,0
8,1,0,34.0,100.0,70.0,0,0,1,1,0,0,1,0,0,0
9,1,0,34.0,110.0,70.0,0,0,1,0,0,1,0,0,1,1


In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 300
hidden_nodes_layer3 = 100
hidden_nodes_layer4 = 50
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               7500      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               150300    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 193,001
Trainable params: 193,001
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1601/1601 [==============================] - 3s 1ms/step - loss: 0.5688 - accuracy: 0.7213
Epoch 2/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5634 - accuracy: 0.7241
Epoch 3/100
1601/1601 [==============================] - 2s 2ms/step - loss: 0.5618 - accuracy: 0.7257
Epoch 4/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5610 - accuracy: 0.7254
Epoch 5/100
1601/1601 [==============================] - 3s 2ms/step - loss: 0.5599 - accuracy: 0.7263
Epoch 6/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5595 - accuracy: 0.7261
Epoch 7/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5590 - accuracy: 0.7263
Epoch 8/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5591 - accuracy: 0.7263
Epoch 9/100
1601/1601 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7264
Epoch 10/100
1601/1601 [==============================] - 2s 1ms/step - l

1601/1601 [==============================] - 2s 1ms/step - loss: 0.5278 - accuracy: 0.7423
Epoch 80/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5294 - accuracy: 0.7425
Epoch 81/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5287 - accuracy: 0.7418
Epoch 82/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5286 - accuracy: 0.7431
Epoch 83/100
1601/1601 [==============================] - 2s 2ms/step - loss: 0.5289 - accuracy: 0.7419
Epoch 84/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5267 - accuracy: 0.7427
Epoch 85/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5279 - accuracy: 0.7430
Epoch 86/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5262 - accuracy: 0.7429
Epoch 87/100
1601/1601 [==============================] - 2s 1ms/step - loss: 0.5264 - accuracy: 0.7437
Epoch 88/100
1601/1601 [==============================] - 2s 1ms/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {(model_loss):.3f}, Accuracy: {(model_accuracy):.3f}")

534/534 - 0s - loss: 0.6620 - accuracy: 0.7164
Loss: 0.662, Accuracy: 0.716


In [21]:
print(f"The accuracy is 72% after adding additional neurons to each of the 4 hidden layers")

The accuracy is 72% after adding additional neurons to each of the 4 hidden layers
